The Python API package is called ee. It must be imported and initialized for each new Python session and script.
Authenticate to the Earth Engine servers

In [ ]:
import ee

ee.Authenticate()

Initialize the API

In [ ]:
try:    
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

In [3]:
import ipywidgets as widgets
from ipyleaflet import *
from ipywidgets import jslink
from geemap.utils import *
from IPython.display import display
import geemap

Map = geemap.Map(height='800px')

Import the shape file of the flooded districts in Punjab

In [4]:
floodedDistricts = ee.FeatureCollection('users/namratas/FloodedPunjabdistricts')

In [5]:
Map.setOptions('TERRAIN')
Map.centerObject(floodedDistricts, 8)

# Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte()

# Paint both the fill and the edges.
filledOutlines = empty.paint(floodedDistricts, 'BIOME_NUM').paint(floodedDistricts, 0, 2)
Map.addLayer(filledOutlines, {"palette": 'BEBEBE', max: 14}, 'ROI')

Define the time periods

In [6]:
beforeStart = '2019-03-13'
beforeEnd = '2019-06-13'

afterStart = '2019-08-21'
afterEnd = '2019-09-16'

In [7]:
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")

s1Collection = s1.filter(ee.Filter.eq('instrumentMode', 'IW'))\
                      .filter(ee.Filter.eq('transmitterReceiverPolarisation', ['VV','VH'])) \
                      .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
                      .filter(ee.Filter.eq('resolution_meters',10)) \
                      .filterBounds(floodedDistricts) \
                      .select('V.*')

In [8]:
before_collection = s1Collection.select('VH').filterDate(beforeStart, beforeEnd)
after_collection = s1Collection.select('VH').filterDate(afterStart, afterEnd)

before = before_collection.mosaic().clip(floodedDistricts)
after = after_collection.mosaic().clip(floodedDistricts)

Refined Lee Speckle Filter for flood assessment, as coded in the SNAP 3.0 S1TBX
https://github.com/senbox-org/s1tbx/blob/master/s1tbx-op-sar-processing/src/main/java/org/esa/s1tbx/sar/gpf/filtering/SpeckleFilters/RefinedLee.java and adapted by Guido Lemoine

In [9]:
# Function to convert from dB
def toNatural(img):
    return ee.Image(10.0).pow(img.select(0).divide(10.0))

# Function to convert to dB
def toDB(img):
    return ee.Image(img).log10().multiply(10.0)

def RefinedLee(img):
    # img must be in natural units, i.e. not in dB!
    # Set up 3x3 kernels 
    weights3 = ee.List.repeat(ee.List.repeat(1,3),3)
    kernel3 = ee.Kernel.fixed(3,3, weights3, 1, 1, False)
    
    mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3)
    variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3)

    # Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
    sample_weights = ee.List([[0,0,0,0,0,0,0], [0,1,0,1,0,1,0],\
                             [0,0,0,0,0,0,0], [0,1,0,1,0,1,0], \
                             [0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0]])

    sample_kernel = ee.Kernel.fixed(7,7, sample_weights, 3,3, False)

    # Calculate mean and variance for the sampled windows and store as 9 bands
    sample_mean = mean3.neighborhoodToBands(sample_kernel)
    sample_var = variance3.neighborhoodToBands(sample_kernel)

    # Determine the 4 gradients for the sampled windows
    gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs()
    gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
    gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
    gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())

    # And find the maximum gradient amongst gradient bands
    max_gradient = gradients.reduce(ee.Reducer.max())

    # Create a mask for band pixels that are the maximum gradient
    gradmask = gradients.eq(max_gradient)

    # duplicate gradmask bands: each gradient represents 2 directions
    gradmask = gradmask.addBands(gradmask)

    # Determine the 8 directions
    directions = sample_mean.select(1) \
                            .subtract(sample_mean.select(4)) \
                            .gt(sample_mean.select(4).subtract(sample_mean.select(7))) \
                            .multiply(1)
    directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)) \
                           .gt(sample_mean.select(4).subtract(sample_mean.select(2))) \
                           .multiply(2))
    directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)) \
                           .gt(sample_mean.select(4).subtract(sample_mean.select(5))) \
                           .multiply(3))
    directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)) \
                           .gt(sample_mean.select(4).subtract(sample_mean.select(8))) \
                           .multiply(4))
    # The next 4 are the not() of the previous 4
    directions = directions.addBands(directions.select(0).Not().multiply(5))
    directions = directions.addBands(directions.select(1).Not().multiply(6))
    directions = directions.addBands(directions.select(2).Not().multiply(7))
    directions = directions.addBands(directions.select(3).Not().multiply(8))

    # Mask all values that are not 1-8
    directions = directions.updateMask(gradmask)

    # "collapse" the stack into a singe band image (due to masking, each pixel has just 
    # one value (1-8) in it's directional band, and is otherwise masked)
    directions = directions.reduce(ee.Reducer.sum()) 

    pal = ['ffffff','ff0000','ffff00', '00ff00', '00ffff', '0000ff', 'ff00ff', '000000']
    # Map.addLayer(directions.reduce(ee.Reducer.sum()), {min:1, max:8, palette: pal}, 'Directions', false)

    sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))

    # Calculate localNoiseVariance
    sigmaV = sample_stats.toArray().arraySort().arraySlice(0,0,5).arrayReduce(ee.Reducer.mean(), [0])

    # Set up the 7*7 kernels for directional statistics
    rect_weights = ee.List.repeat(ee.List.repeat(0,7),3).cat(ee.List.repeat(ee.List.repeat(1,7),4))

    diag_weights = ee.List([[1,0,0,0,0,0,0], [1,1,0,0,0,0,0],\
                                [1,1,1,0,0,0,0], [1,1,1,1,0,0,0], \
                                [1,1,1,1,1,0,0], [1,1,1,1,1,1,0], [1,1,1,1,1,1,1]])

    rect_kernel = ee.Kernel.fixed(7,7, rect_weights, 3, 3, False)
    diag_kernel = ee.Kernel.fixed(7,7, diag_weights, 3, 3, False)

    # Create stacks for mean and variance using the original kernels. Mask with relevant direction.
    dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
    dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))

    dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)))
    dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)))

    # and add the bands for rotated kernels
    #for  (var i=1; i<4; i++):
    for i in range(1, 4):
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)) \
                                        .updateMask(directions.eq(2*i+1)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)) \
                                        .updateMask(directions.eq(2*i+1)))
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)) \
                                        .updateMask(directions.eq(2*i+2)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)) \
                                        .updateMask(directions.eq(2*i+2)))  

    # "collapse" the stack into a single band image (due to masking, each pixel has just one value in it's directional band, and is otherwise masked)
    dir_mean = dir_mean.reduce(ee.Reducer.sum())
    dir_var = dir_var.reduce(ee.Reducer.sum())

    # A finally generate the filtered value
    varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))

    b = varX.divide(dir_var)

    result = dir_mean.add(b.multiply(img.subtract(dir_mean)))
    
    return(result.arrayFlatten([['sum']]))


Get before-flood and after-flood Refined Lee filtered images

In [10]:
before_rLee = ee.Image(toDB(RefinedLee(toNatural(before))))
after_rLee = ee.Image(toDB(RefinedLee(toNatural(after))))

Calculate the flood extent. Create a flood extent mask using a threshold value of 1.25

In [11]:
difference_rLee = after_rLee.divide(before_rLee)

DIFF_UPPER_THRESHOLD = 1.25

# Identify pixels above the threshold (i.e flood pixels).
# Set all other pixels to 0
floodMask = difference_rLee.gt(DIFF_UPPER_THRESHOLD).rename('water') 

# Keep only the flood pixels. Remove all pixels equal to 0
floodedAreas = floodMask.selfMask()

Include JRC layer on surface water seasonality and occurrence to mask flood pixels from
areas of "permanent" water, i.e., where there is water > 10 months of the year

In [12]:
jrcGSW = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
waterBodies = jrcGSW.select('occurrence').clip(floodedDistricts)
vis_waterBodies = {min:0, max:100, "palette": '#0D0887'}
          
Map.addLayer(**{
    # mask waterBodies so as to not detect flood in the waterBodies.
    # .divide(100) causes the opacity/transparency of the pixels to
    # be set based on the waterBodies occurrence value.
    
    'ee_object': waterBodies.updateMask(waterBodies.divide(100)),\
    'name': "Permanent water bodies (BLUE)", \
    'vis_params': vis_waterBodies    
})

Set a mask = 1 if water > 10 months of the year, i.e permanent water pixels

Non-permanent water (flood) pixels = 0

Remove flood pixels

Identify flooded areas that are a minimum 0.25 ha (>= 3 pixels)

connectedPixelCount() to get a contiguous area

In [13]:
seasonality = jrcGSW.select('seasonality')        

permWater = seasonality.gte(10).selfMask()
  
# In the floodedAreas layer, assign all permanent water pixels = 0
permWater_removed = floodedAreas.where(permWater,0)
  
# Remove all the permanent water present in the flooded layer.
# floodedAreas layer has only flood pixels
onlyFlooded = floodedAreas.updateMask(permWater_removed).selfMask()
 
# Set minimum flood area in pixels = 3, approximately 0.25 ha
minFloodPixels = ee.Number(3)
  
# Scale the results to display on the map. 
# Reprojecting with a specified scale ensures that the pixel area 
# does not change with zoom.
prj = jrcGSW.projection()
scale = prj.nominalScale()
contFloodArea = onlyFlooded.connectedPixelCount(25) \
                      .reproject(prj.atScale(scale))
                      
# Apply the minimum area requirement.
minFloodArea = contFloodArea.gte(minFloodPixels).selfMask() \
                    .reproject(prj.atScale(scale))

Mask out areas with more than 5 percent slope using the DEM HydroSHEDS (computing gradients requires a fixed projection)

In [14]:
demHydroSHEDS = ee.Image("WWF/HydroSHEDS/03VFDEM")
terrain = ee.Algorithms.Terrain(demHydroSHEDS)
slope = terrain.select('slope')
finalFloodedArea = minFloodArea.updateMask(slope.lt(5))

Import as an Asset - imp_finalFloodedArea

In [15]:
imp_finalFloodedArea = ee.Image("users/namratas/FinalFloodedArea")

# Display the flooded areas
Map.addLayer(imp_finalFloodedArea.clip(floodedDistricts), {'palette':"red", 'opacity': 0.99}, 'Flooded areas (RED)', 1);


Calculate the total flooded area (ha)

Use pixelArea() to get the value of each pixel in square metres, and sum over the result for a measure of area.

In [16]:
floodedArea = imp_finalFloodedArea.multiply(ee.Image.pixelArea())

floodedAreaSize = floodedArea.reduceRegion( **{
      'reducer': ee.Reducer.sum(),
      'geometry': floodedDistricts.geometry(),
      'scale': 30, 
      'maxPixels': 1e13
})

# Convert the floodedAreaSize to hectares (area calculations are originally given in meters)
# divide by 10,000 to convert to hectare
floodedAreaSize = floodedAreaSize.getNumber('water') \
                        .divide(10000) \
                        .round()

Calculate the area (ha) of flooded agricultural land 

Identify affected areas that are a minimum 0.5 ha (6 pixels)

connectedPixelCount() to get a contiguous area

In [17]:
imp_gfsad30_Cropland = ee.Image("users/namratas/gfsad30_Cropland_Dataset") 
gfsad30 = imp_gfsad30_Cropland.clip(floodedDistricts)

# Create a raster showing affected agricultural area in the flood layer
onlyFloodedAgri = imp_finalFloodedArea.updateMask(gfsad30).selfMask()

Map.addLayer(onlyFloodedAgri.select('water'), {'palette': '#2B5329'}, 'Affected Agricultural Land (GREEN)',1)

prj = gfsad30.projection()
scale = prj.nominalScale()
  
# Use connectedPixelCount() to get a contiguous area
contFloodedAgriArea = onlyFloodedAgri.connectedPixelCount(25) \
                      .reproject(prj.atScale(scale))
                      
# Set minimum agricultural area = 6 pixels, i.e approximately 0.5 ha
minAgriPixels = ee.Number(6)
  
floodedAgriArea = contFloodedAgriArea.gte(minAgriPixels).selfMask() \
                    .reproject(prj.atScale(scale))

In [18]:
affectedAgriArea = onlyFloodedAgri.multiply(ee.Image.pixelArea())
  
totalAffectedAgriLand = affectedAgriArea.reduceRegion( **{
      'reducer': ee.Reducer.sum(),
      'geometry': floodedDistricts.geometry(),
      'scale': 30, 
      'maxPixels': 1e13
})
  
# Convert the totalAffectedAgriLand to hectares (area calculations are originally given in meters)
# divide by 10,000 to convert to hectare
flooded_agriArea_hectares = ee.Number(totalAffectedAgriLand.get('water')) \
                                  .divide(10000) \
                                  .round()

Calculate the population in the flooded areas

In [ ]:
popDataset = ee.ImageCollection("WorldPop/POP") 
  
punjabPop  = popDataset \
             .filter(ee.Filter.equals('UNadj', 'no')) \
             .filter(ee.Filter.equals('year', 2015)) \
             .mosaic() \
             .clip(floodedDistricts)

# Create a raster showing affected population using the flood layer
affectedPopMask = punjabPop.updateMask(finalFloodedArea).selfMask()

Import as an Asset - imp_affectedPop

In [21]:
imp_affectedPop = ee.Image("users/namratas/affectedPunjabPopulation") 

# calculate stats
affectedPop = imp_affectedPop.reduceRegion( **{
    'reducer': ee.Reducer.sum(),
    'geometry': floodedDistricts,
    'scale': 100,
    'maxPixels': 1e15
  })
  
totalAffectedPop = ee.Number(affectedPop.get('population')).toInt()

Visualisation with geemap, ipyleaflet and ipywidgets

In [22]:
# Set up all the necessary widgets
layout = widgets.Layout(display='flex',\
                            flex_flow='column',\
                            align_items='stretch',\
                            border='solid'
                            )

style = {'description_width': 'initial'}

floodedAreaHTML = widgets.HTML(
    value = f"<b><font color='bf0f19'>{str(floodedAreaSize.getInfo())}</b>",
    placeholder='',
    description= '<b>Flooded Area (in ha) - </b>',
    style = style
)

floodedAgriAreaHTML = widgets.HTML(
    value= f"<b><font color='bf0f19'>{str(flooded_agriArea_hectares.getInfo())}</b>", 
    placeholder=' ',
    description='<b>Flooded Agricultural Area (in ha) - </b>',
    style = style,
)

affectedPopHTML = widgets.HTML(
    value= f"<b><font color='bf0f19'>{str(totalAffectedPop.getInfo())}</b>",
    placeholder=' ',
    description='<b>Affected population - </b>',
    style = style,
)

credits = widgets.HTML(
    value='<p style="font-size:90%;color:grey"><b>Credits: </b>Ujaval Gandhi, Founder<a href="https://spatialthoughts.com/"> Spatial Thoughts</a></p>',
    placeholder='',
    descripition='Credits:'
)

zoom_slider = widgets.IntSlider(
    description='Zoom level:', 
    min=0, 
    max=15, 
    value=8
)

jslink((zoom_slider, 'value'), (Map, 'zoom'))

box = widgets.VBox([floodedAreaHTML, floodedAgriAreaHTML, affectedPopHTML], layout=layout)


Add the widgets to the map

In [23]:
results = WidgetControl(widget=box, position='topleft')
ujaval = WidgetControl(widget=credits, position='bottomleft')
zoom = WidgetControl(widget=zoom_slider, position='topright')
layers_control = LayersControl(position='topright')

legend_dict = {
    'flooded areas': 'EB0000',
    'affected agricultural area': '2B5329',
    'permanent water bodies': '0D0887'
}

Map.clear_controls()

Map.add_legend(legend_title = 'Legend',legend_dict = legend_dict)
Map.add_control(results)
Map.add_control(ujaval)
Map.add_control(zoom)

Map.add_control(FullScreenControl())
Map.add_control(layers_control)

Map

Map(center=[30.946658235946547, 75.4731802404629], controls=(WidgetControl(options=['position'], position='bot…